In [1]:
# ********** Import Statements ************************************************************ #
import sys
import spotipy
import spotipy.util as util

import pandas as pd

In [ ]:
# ********** Variables ******************************************************************** #
self_username = '1242091675'
other_username = '12166088827'
#
#------#

scope = 'user-library-read'
client_id = 'a7a7510a67034ce28856364ce96c5504'
client_secret = '0c6ce5bb04264146aa6ed2f6842a83a7'
redirect_uri = 'http://localhost:1410/'

In [ ]:
# ********** Spotify Connection String **************************************************** #
token = util.prompt_for_user_token(username,
                                   scope,
                                   client_id,
                                   client_secret,
                                   redirect_uri)
sp = spotipy.Spotify(auth=token)

In [ ]:
# # ********** Test Class ******************************************************************* #
# class spootifer(sp_id):
#     def __init__(self, sp_id):
#         self.id = sp_id
        
    

In [ ]:
other_playlists['items'][0]#.keys()#['id']

In [ ]:
# ********** Functions ******************************************************************** #
def show_tracks(tracks):
    for i, item in enumerate(tracks['items']):
        track = item['track']
        print "   %d %32.32s %s" % (i, track['artists'][0]['name'],
            track['name'])
        
def get_playlist_songs_into_df(usr, playlists, name=''):
    songs_df = pd.DataFrame()
    
    def parse_playlist(playlist):
        temp_df = pd.DataFrame()
        if playlist['owner']['id'] == usr:
            results = sp.user_playlist(usr, playlist['id'],
                fields="tracks,next")
            tracks = results['tracks']
            for i, item in enumerate(tracks['items']):
                track = item['track']

                track_deets = pd.DataFrame({'playlist_id': playlist['id'],
                                           'playlist': playlist['name'], 
                                           'artist': track['artists'][0]['name'], 
                                           'title': track['name'],
                                           'song_id': track['id']})
                song_deets = pd.DataFrame(sp.audio_features(track['id']))
                playlist_deets = pd.merge(track_deets, song_deets, 
                                          how='left', 
                                          left_on = 'song_id', 
                                          right_on='id')
                temp_df = temp_df.append(playlist_deets)
                
        return temp_df

    if name != '':
        playlist = [x for x in other_playlists['items'] if x['name'] == name][0]
        pl = parse_playlist(playlist)
        songs_df = songs_df.append(pl)
    else:
        for playlist in playlists['items']:
            songs_df = songs_df.append(parse_playlist(playlist), ignore_index=True)
    return songs_df
            #print tracks['artists'][0]['name'] + ' ' + track['name']
    
def get_saved_tracks_into_df():
    col_names = ['Artist', 'Title']
    songs_df = pd.DataFrame(columns=col_names)
    
    # we have this but just incase:
    saved_tracks = sp.current_user_saved_tracks()
    
    for item in saved_tracks['items']:
        track = item['track']
        
        song_deets = [[track['artists'][0]['name'], track['name']]]
        songs_df = songs_df.append(pd.DataFrame(song_deets, columns=col_names), ignore_index=True)
        
    return songs_df

def get_playlist_names(playlists):
    return [x['name'] for x in other_playlists['items']]

def get_playlist_id(playlists, name):
    return [x['id'] for x in other_playlists['items'] if x['name'] == 'Mr. Disco Pop Pants'][0]

In [ ]:
# ********** Assorted Results ************************************************************* #
user = sp.user(self_username)
user_saved_tracks = sp.current_user_saved_tracks()
user_playlists = sp.user_playlists(self_username)
other = sp.user(other_username)
other_playlists = sp.user_playlists(other_username)
 

In [ ]:
get_playlist_names(other_playlists)

In [ ]:
get_playlist_id(other_playlists, 'Mr. Disco Pop Pants')

In [ ]:
other_playlists_df = get_playlist_songs_into_df(other_username, other_playlists, 'sexy sexy')

In [ ]:
other_playlists_df

In [ ]:
print results['items'][0]['track']['name']

In [ ]:
# Use '#polycomm'
# Over time, are there any trends.
# Are there any shared trends?

In [ ]:
get_playlist_names(other_playlists)

In [ ]:
results['items'][0]['track']['id']

In [ ]:
results = sp.current_user_saved_tracks(limit=50, offset=0)
for item in results['items']:
    track = item['track']
    print track['name'] + ' - ' + track['artists'][0]['name']


In [ ]:
col_names = ['Artist', 'Title']
df = pd.DataFrame(columns=col_names)

vector = [['a', 'b']]
vector2 = [['c', 'd']]
df = df.append(pd.DataFrame(vector, columns=col_names), ignore_index=True)
df.append(pd.DataFrame(vector2, columns=col_names), ignore_index=True)


In [ ]:
songs_cassie_df = get_playlist_songs_into_df(username, playlists_cassie)
songs_cassie_df = songs_cassie_df.append(get_saved_tracks_into_df())

In [ ]:
songs_alex_df = get_playlist_songs_into_df('alex-soto', playlists_alex)

In [ ]:
songs_dwayne_df = get_playlist_songs_into_df('dwayne.pryce', playlists_dwayne)

In [ ]:
s1 = pd.merge(songs_dwayne_df, songs_alex_df, how='inner', on=['Artist', 'Title'])
s1